# Sentinel Devices - Anomaly Detection Project

ISYE / CSE / MGT 6748

Student: Tung Nguyen, tnguyen844@gatech.edu

This notebook was run on: Python 3.13, macOS 15 Sequoia, Macbook Pro (M1 Max)


## Database Setup & Model Training

Before running this Jupyter Notebook to set up and train the anomaly detection models, set up the Python environment using the commands below:

```bash
# Create virtual environment
python -m venv ./sentinel-devices

# Install dependencies
pip install -r requirements.txt

# Optional: Upgrade pip
pip install --upgrade pip
```

Then set the kernel to the python environment that was just set up.

After running this notebook, there should be:
* A SQLite3 database with training data, prediction data fully set up in the `02-data` folder
* A ChromaDB database with the MetroPT Research Paper loaded in the `02-data` folder
* 8 trained Prophet models in the `03-models` folder

In [ ]:
from dotenv import load_dotenv

import os
import sqlite3
import pandas as pd
from datetime import datetime, timedelta
from tqdm import tqdm
from joblib import dump, load
from prophet import Prophet
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_chroma import Chroma

load_dotenv(r'../.env');
print('OPENAI_API_KEY' in os.environ)

analog_sensors = [
    'tp2',
    'tp3',
    'h1',
    'dv_pressure',
    'reservoirs',
    'oil_temperature',
    'flowmeter',
    'motor_current'
]

digital_sensors = [
    'comp',
    'dv_electric',
    'towers',
    'mpg',
    'lps',
    'pressure_switch',
    'oil_level',
    'caudal_impulses'
]

gps_sensors = [
    'gps_long',
    'gps_lat',
    'gps_speed',
    'gps_quality'
]

## Global Settings

This section controls how much training data is used, and which specific part of the training data is used.

This section also contains global model parameters that control how anomalies are flagged and consolidated.

In [ ]:
today = datetime(2022, 2, 28, 2, 0, 0) # Simulate "today" as Feb 27th

training_window = 30*2 # days
inference_window = 7   # days

n_signal_threshold = 5     # How many sensors need to be out-of-bounds at the same time for the model to flag the timestamp as an anomaly?
out_of_bounds_buffer = 0.1 # How far out-of-bounds does the signal need to be before its flagged as an anomaly?

event_consolidation_threshold_secs = 10 # Max # of seconds between consecutive events allowed, otherwise, the two events are conslidated into one.

### Calculated Fields

In [38]:
train_start = (today - timedelta(days=training_window)).strftime(r'%Y-%m-%d %H:%M:%S')
inference_start = (today - timedelta(days=inference_window)).strftime(r'%Y-%m-%d %H:%M:%S')
end = today.strftime(r'%Y-%m-%d %H:%M:%S')

print(f'Train Start: {train_start} - {end}')
print(f'Inference Period: {inference_start} - {end}')

Train Start: 2021-12-30 02:00:00 - 2022-02-28 02:00:00
Inference Period: 2022-02-21 02:00:00 - 2022-02-28 02:00:00


## Create SQLite Database
Load data from CSV into a SQLite3 database.

In [34]:
%%time

# Column name remapping
config = [
    ('timestamp', 'ts'),
    ('TP2', 'tp2'),
    ('TP3', 'tp3'),
    ('H1', 'h1'),
    ('DV_pressure', 'dv_pressure'),
    ('Reservoirs', 'reservoirs'),
    ('Oil_temperature', 'oil_temperature'),
    ('Flowmeter', 'flowmeter'),
    ('Motor_current', 'motor_current'),
    ('COMP', 'comp'),
    ('DV_eletric', 'dv_electric'),
    ('Towers', 'towers'),
    ('MPG', 'mpg'),
    ('LPS', 'lps'),
    ('Pressure_switch', 'pressure_switch'),
    ('Oil_level', 'oil_level'),
    ('Caudal_impulses', 'caudal_impulses'),
    ('gpsLong', 'gps_long'),
    ('gpsLat', 'gps_lat'),
    ('gpsSpeed', 'gps_speed'),
    ('gpsQuality', 'gps_quality')
]

with sqlite3.connect(r'./02-data/data.db') as con:

    # Load data from CSV -> SQLite
    for csv_file in ['./raw-data/train_data_part1.csv', './raw-data/train_data_part2.csv']:
        data = pd.read_csv(csv_file, parse_dates=['timestamp'], chunksize=100_000)
        for chunk in data:
            chunk.to_sql('train_data', con, if_exists='append', index=False)

    cur = con.cursor()

    # Rename columns
    for c in config:
        cur.execute(f'alter table train_data rename column {c[0]} to {c[1]};')
    
    # Add indices & columns
    cur.execute('create index if not exists idx_ts on train_data(ts)')
    cur.execute('alter table train_data add column failure_id integer')
    cur.execute('alter table train_data add column pseudo_label real;')

    # Load Failure ID column
    cur.execute("update train_data set failure_id = 1 where '2022-02-28 21:53:00' <= ts and ts < '2022-03-01 02:00:00';")
    cur.execute("update train_data set failure_id = 2 where '2022-03-23 14:54:00' <= ts and ts < '2022-03-23 15:24:00';")
    cur.execute("update train_data set failure_id = 3 where '2022-05-30 12:00:00' <= ts and ts < '2022-06-02 06:18:00';")

    # Load Pseudo-Label column
    cur.execute("update train_data set pseudo_label = 1 where '2022-02-21 06:00:00' <= ts and ts < '2022-02-28 02:00:00';")
    cur.execute("update train_data set pseudo_label = 2 where '2022-03-16 06:00:00' <= ts and ts < '2022-03-23 02:00:00';")
    cur.execute("update train_data set pseudo_label = 3 where '2022-05-23 06:00:00' <= ts and ts < '2022-05-30 02:00:00';")

CPU times: user 41 s, sys: 6.38 s, total: 47.4 s
Wall time: 49.1 s


## Create Vector Database
Load PDF text into a vector database.

In [36]:
%%time

# Load PDF
# There's only 1 PDF but theoretically, this code can be extended to multiple PDFs
loader = PyPDFLoader(r'./01-docs/s41597-022-01877-3.pdf')
docs = loader.load()

# Split PDFs into chunks
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
    add_start_index=True
)
splits = text_splitter.split_documents(docs)

# Generate & save Vector Database
embeddings = OpenAIEmbeddings(model='text-embedding-3-large')
vector_store = Chroma(
    collection_name='train_metadata',
    embedding_function=embeddings,
    persist_directory='./02-data/chroma_db'
)

ids = vector_store.add_documents(splits)
print(ids)

['be5ac3ef-1cac-4849-b4fc-3bb826497d52', '13e9cb95-4b6e-4e93-acb0-c720d31d7bab', 'bb87c475-0e08-4b06-a651-fe05f95ac343', '03808692-57cc-4fa4-ad0b-d3d62e7258c3', '4f594858-6fb8-40ff-ae76-2071ef95dd3a', '32a67021-bd19-48a9-8700-3c4472caaf95', '180145ef-4b54-42c9-a4f1-1204ccfdfabc', '1e5914c9-cf32-4dc5-b68d-11d48e02439e', '36e0602b-ff8e-4a39-9ba8-2da494cab0f0', 'af279618-b1ad-4399-9ef7-7cd3c653d7f1', 'ecaa39d8-2336-4e6e-929c-63d4adb2235c', '192dfe39-2d8d-402e-8918-a442269eb5b7', 'b444115c-74dc-4749-9fcc-456342497265', '33fe7c0a-f55f-450e-9ae0-a5910039ba6d', 'c27092bb-ff81-4a82-a14a-ce1bc2abfce8', 'e6f16af0-20b5-4bef-872a-72fe7ad65ffb', 'f65df40a-dc1d-4805-b18b-f0bd79dca54d', '4201c19b-b330-45f8-a679-c0610bd82f71', '48ca4518-6ab5-4971-a24c-b6ffad73ddac', 'b150a373-2ee9-418c-9c9c-876ce007895a', 'e464fb45-963a-42da-b08e-3644c6ebfb5b', 'fb41a688-2fa0-4086-b603-1c1618c3889d', 'b073819c-d0b8-4ebf-a293-d4b9393b041a', '67cd58ed-bc84-40cf-9f68-c9c42af7d29c', '2294be9d-9978-451a-8ee5-7ca05a6a3d3c',

## Train Models
Train Prophet models (sequentially)

In [ ]:
with sqlite3.connect(r'./02-data/data.db') as con:
    df = pd.read_sql(f"""
        select * 
        from train_data
        where 
            failure_id is null
            and '{train_start}' <= ts and ts < '{end}'
    """, con, parse_dates=['ts'])

print(df.shape)

(4175003, 23)


In [ ]:
for sensor in tqdm(analog_sensors):
    model = Prophet(
        changepoint_prior_scale=0.5, # Lower changepoint prior
        yearly_seasonality=False,    # Use custom seasonalities instead 
        weekly_seasonality=False,
        daily_seasonality=False
    )

    model.add_seasonality(name='30min', period=60*30, fourier_order=5) # Split day in 30min intervals
    model.add_seasonality(name='daily', period=72000, fourier_order=5) # 6am - 2am

    model.fit(df.loc[:, ['ts', sensor]].rename(columns={'ts': 'ds', sensor: 'y'}))

    dump(model, f'03-models/prophet_{sensor}.joblib')

  0%|          | 0/8 [00:00<?, ?it/s]15:10:00 - cmdstanpy - INFO - Chain [1] start processing
15:22:34 - cmdstanpy - INFO - Chain [1] done processing
 12%|█▎        | 1/8 [14:25<1:41:01, 865.87s/it]15:24:16 - cmdstanpy - INFO - Chain [1] start processing
15:33:25 - cmdstanpy - INFO - Chain [1] done processing
 25%|██▌       | 2/8 [25:17<1:13:59, 739.92s/it]15:35:08 - cmdstanpy - INFO - Chain [1] start processing
15:52:45 - cmdstanpy - INFO - Chain [1] done processing
 38%|███▊      | 3/8 [44:36<1:17:36, 931.31s/it]15:54:26 - cmdstanpy - INFO - Chain [1] start processing
15:55:17 - cmdstanpy - INFO - Chain [1] done processing
 50%|█████     | 4/8 [47:09<41:35, 623.83s/it]  15:56:58 - cmdstanpy - INFO - Chain [1] start processing
16:19:40 - cmdstanpy - INFO - Chain [1] done processing
 62%|██████▎   | 5/8 [1:11:31<46:18, 926.33s/it]16:21:21 - cmdstanpy - INFO - Chain [1] start processing
16:39:38 - cmdstanpy - INFO - Chain [1] done processing
 75%|███████▌  | 6/8 [1:31:30<33:57, 1018.94s

## Save Predictions
Load model predictions into SQLite3.

In [48]:
chunk_size = 1_000_000

with sqlite3.connect(r'./02-data/data.db') as con:
    cur = con.cursor()

    # Create results table
    cur.execute('drop table if exists results;') 
    cur.execute('''
        create table results (
            ts timestamp,
            sensor text,
            yhat real,
            yhat_lower real,
            yhat_upper real,
            yhat_lower_with_buffer real,
            yhat_upper_with_buffer real,
            pred integer
        );
    ''')

    # Calculate inference df
    inference_df = pd.read_sql(f"""
        select * 
        from train_data
        where 
            '{inference_start}' <= ts and ts < '{end}'
    """, con, parse_dates=['ts'])

    # Load model and predict
    for sensor in tqdm(analog_sensors):
        model = load(f'03-models/prophet_{sensor}.joblib')

        for start in range(0, len(inference_df), chunk_size):
            chunk = inference_df.iloc[start:start + chunk_size]

            pred_df = (
                model
                    .predict(chunk.loc[:, ['ts', sensor]].rename(columns={'ts': 'ds'}))
                    .merge(chunk.loc[:, ['ts', sensor]], left_on='ds', right_on='ts')
            )

            pred_df['sensor'] = sensor
            pred_df['range'] = pred_df['yhat_upper'] - pred_df['yhat_lower']
            pred_df['range_expanded'] = pred_df['range'] * (1 + out_of_bounds_buffer)
            pred_df['yhat_lower_with_buffer'] = pred_df['yhat'] - pred_df['range_expanded'] / 2
            pred_df['yhat_upper_with_buffer'] = pred_df['yhat'] + pred_df['range_expanded'] / 2

            pred_df['pred'] = (
                (pred_df[sensor] < pred_df['yhat_lower_with_buffer']) |
                (pred_df[sensor] > pred_df['yhat_upper_with_buffer'])
            ).astype(int)

            columns = [
                'ts',
                'sensor',
                'yhat',
                'yhat_lower',
                'yhat_upper',
                'yhat_lower_with_buffer',
                'yhat_upper_with_buffer',
                'pred'
            ]
            
            pred_df[columns].to_sql(
                name='results',
                con=con, 
                if_exists='append', 
                index=False,
                chunksize=999,
                method='multi'
            )

100%|██████████| 8/8 [05:51<00:00, 43.99s/it]


## Post-Process Predictions
### Aggregate Predictions
Individual model results from the previous section are loaded into a flat table. This section pivots the data in order to calculate the final prediction.

In [98]:
%%time

with sqlite3.connect(r'./02-data/data.db') as con:
    cur = con.cursor()

    cur.execute('drop table if exists results_agg;')

    cur.execute("""create table if not exists results_agg (ts timestamp, {cols}, {agg_cols})""".format(
        cols = ', '.join([f'{sensor}_pred integer' for sensor in analog_sensors]),
        agg_cols = 'total_sum integer, pred integer'
    ))

    cur.execute('''
        insert into results_agg (ts, {cols})
        select ts, {sum_cols}
        from results
        group by ts
    '''.format(
        cols = ', '.join([f'{sensor}_pred' for sensor in analog_sensors]),
        sum_cols = ', '.join([f"sum(case when sensor='{sensor}' then pred else 0 end) as {sensor}_pred" for sensor in analog_sensors])
    ))

    cur.execute('''update results_agg set total_sum = {total_sum_col}'''.format(
        total_sum_col = ' + '.join([f'{sensor}_pred' for sensor in analog_sensors])
    ))

    cur.execute(f'''update results_agg set pred = case when total_sum >= {n_signal_threshold} then 1 else 0 end''')

CPU times: user 2.41 s, sys: 374 ms, total: 2.79 s
Wall time: 2.84 s


### Event Consolidation
This section consolidates short events, e.g. you can have two 5 second events that are 4 seconds apart. This section would consolidate those two events into a single 14 second event in order to reduce the amount of noise in the predictions.

In [99]:
%%time

with sqlite3.connect(r'./02-data/data.db') as con:
    cur = con.cursor()

    # Create initial anomalies table
    cur.execute('drop table if exists anomalies')
    cur.execute("""
        create table anomalies (
            event_id int,
            start_ts timestamp,
            end_ts timestamp,
            event_duration_in_secs int
        )
    """)

    cur.execute(f"""
        with 
        lagged as (
            select
                ts
                ,pred
                ,lag(pred, 1, 0) over (order by ts asc) as prev_pred
            from results_agg
        ),
        flagged as (
            select
                ts
                ,pred
                ,sum(case when pred != prev_pred then 1 else 0 end) over (order by ts rows unbounded preceding) as event_id
            from lagged
        )
        insert into anomalies (event_id, start_ts, end_ts, event_duration_in_secs)
        select
            event_id
            ,min(ts) as start_ts
            ,max(ts) as end_ts
            ,count(*) as event_duration_in_secs
        from flagged
        where pred = 1
        group by event_id
        order by start_ts asc
    """)

    # Consolidate events
    events = pd.read_sql("""
        select 
            *
            ,null as gap
            ,null as new_event_id 
        from anomalies
    """, con, parse_dates=['start_ts', 'end_ts'])

    event_id = 1

    for i in range(events.shape[0]):
        if i == 0:
            events.loc[i, 'new_event_id'] = event_id
            continue

        event_gap = (events.loc[i, 'start_ts'] - events.loc[i - 1, 'end_ts']).seconds - 1
        if event_gap > event_consolidation_threshold_secs:
            event_id += 1

        else:
            events.loc[i, 'gap'] = event_gap

        events.loc[i, 'new_event_id'] = event_id

    events.to_sql(
        name='temp',
        con=con, 
        if_exists='replace', 
        index=False,
        chunksize=999,
        method='multi'
    )

    cur.execute('drop table if exists anomalies_consolidated;')
    cur.execute(f"""
        create table anomalies_consolidated as
            select
                new_event_id as event_id
                ,min(start_ts) as start_ts
                ,max(end_ts) as end_ts
                ,sum(event_duration_in_secs) + sum(gap) as event_duration_in_secs
            from temp
            group by new_event_id
            order by start_ts asc
    """)

    cur.execute('drop table temp;')

CPU times: user 906 ms, sys: 113 ms, total: 1.02 s
Wall time: 1.03 s


### Create Tables for LLM Ingestion
This section creates the tables that are used for `01_inference.ipynb` as well as the Streamlit app in `04-app`.

In [ ]:
with sqlite3.connect(r'./02-data/data.db') as con:
    df = pd.read_sql("""
        select * 
        from anomalies_consolidated 
        where event_duration_in_secs >= 60*5
    """, con, parse_dates=['start_ts', 'end_ts'])

    cur = con.cursor()
    cur.execute('drop table if exists train_data_processed;')
    cur.execute("""
        create table train_data_processed as
            select 
                td.*
                ,ra.tp2_pred
                ,ra.tp3_pred
                ,ra.h1_pred
                ,ra.dv_pressure_pred
                ,ra.reservoirs_pred
                ,ra.oil_temperature_pred
                ,ra.flowmeter_pred
                ,ra.motor_current_pred
                ,ra.total_sum
                ,ra.pred
                ,0 as pred_filtered
                
            from train_data as td
                
                left join results_agg as ra 
                    on td.ts = ra.ts
    """)

    for _, row in df.iterrows():
        cur.execute(f"""
            update train_data_processed
            set pred_filtered = 1
            where '{row['start_ts']}' <= ts and ts <= '{row['end_ts']}';
        """)

    cur.execute('create index if not exists idx_tdp_ts on train_data_processed(ts)')

In [104]:
df

,event_id,start_ts,end_ts,event_duration_in_secs
0,576,2022-02-27 08:53:55,2022-02-27 09:02:06,492
